#### Basic Setup : Building a Basic Rule Based Agent in Python

Installing needed requirements and preprocessing steps.

- Install Requirements for the Notebook.

In [12]:
from IPython.display import clear_output as clr
!python --version
!pip install --upgrade luxai_s2
!pip install importlib-metadata==4.13.0
!pip install gym==0.19
clr()

In [13]:
import os 
os.getcwd()

os.chdir("C:\\Users\\nhw85\\anaconda3\\Lux AI_2\\Lux-Design-S2\\kits\\python")

This step is required since the notebook is running in a local workspace and i dont want to change system paths.

- Copy Lux Project Files to Local Workspace

In [ ]:
# Copying Files to local folder where program is being run...
!cp -r ../input/lux-ai-season-2/* .
clr()

#### Importing Lux Season 2 Environment
- Import all required libraries
- Sample Run : Visualizing the environment as a 2D image

The orange color you see in the image is coming from the sun please dont look at it without sun glasses.

In [ ]:
from luxai_s2.env import LuxAI_S2
import matplotlib.pyplot as plt
import numpy as np
import dataclasses
import json
from luxai_runner.utils import to_json
from IPython import get_ipython
from IPython.display import display, HTML, Javascript
import time
import os
import networkx as nx

In [ ]:
env = LuxAI_S2() # Environment 
obs = env.reset(seed=np.random.randint(1000))
print("Player_names: {}\nObservation Keys: {}".format(list(obs.keys()), list(obs["player_0"].keys())))
img = env.render("rgb_array", width=640, height=640)
plt.figure(figsize = [2,2])
plt.imshow(img)
plt.title("Sample Map Render")
plt.show()

#### Shortest path on Mars using NetworkX (for heavy robots)
I've seen that there is a nice notebook about using Dijstra algorithm to optimize factory placement that I would recommend you to have a look to : https://www.kaggle.com/code/cristojv/moving-units-get-the-rubble-out-of-here

This one is just a short reminder about the "networkx" package that can make shortest path easy if you need to travel on Mars ground. I use it ! Enjoy.

In [ ]:
G = nx.DiGraph()

add_delta = lambda a : tuple(np.array(a[0]) + np.array(a[1]))

rubbles = obs["player_0"]["board"]["rubble"]
for x in range(rubbles.shape[0]):
    for y in range(rubbles.shape[1]):
        G.add_node((x,y), rubble=rubbles[x,y])
print(f"{len(G.nodes)} nodes created.")

deltas = [(1,0),(-1,0),(0,1),(0,-1)]
for g1 in G.nodes:
    x1,y1 = g1
    for delta in deltas:
        g2 = add_delta((g1, delta))
        if G.has_node(g2): 
            G.add_edge(g1, g2, cost=20+rubbles[g2])
print(f"{len(G.edges)} edges created.")

In [ ]:
all_pts = [(x,y) for x in range(0,48) for y in range(0,48)]
np.random.shuffle(all_pts)
ptA, ptB = all_pts[0], all_pts[1]

path = nx.shortest_path(G, source=ptA, target=ptB, weight="cost")

In [ ]:
scale = lambda a: (a+.5)/48*640

plt.figure(figsize=(6,6))
cA = plt.Circle((scale(ptA[0]), scale(ptA[1])), scale(0), color="white")
plt.gca().add_patch(cA)
cB = plt.Circle((scale(ptB[0]), scale(ptB[1])), scale(0), color="yellow")
plt.gca().add_patch(cB)
plt.plot([scale(p[0]) for p in path], [scale(p[1]) for p in path], c="lime")
    
_ = plt.imshow(img, alpha=0.9) 

#### Basic Rule Based Agent


The agent keeps a track of all its bots and opposition bots position and details. In addition it keeps a track of each of its spawned factories.

Early Phase:

- Does not bid for a location to save resources.
- Places the factories at locations where distance from ice and ore is less and rubble density is low, but also keep a distance from opponent's factory to avoid harm from its bots as much as possible.

Regular Phase:

- Each factory is initially assigned bot for each task dig (ice, ore, rubble) and kill enemy bots and keep a track of number of bots per task.
- Heavy bot is assigned task of collecting ice and killing, since ice is very essential in this game.
- For each bot check the closest resources location and move towards them.
- If battery of a bot is too low or enough resources are collected, then move to factory to store the collected items.
- If a collision is predicted in next step avoid moving or move in a different direction.

End Phase:

- Iron man returns to earth after loosing spidy and starts living a normal life. But wait he will create a time machine with an ant and save world.

In [ ]:
%%writefile agent.py 
### Comment above given line to avoid overwriting old agent file ###

from lux.kit import obs_to_game_state, GameState, EnvConfig
from lux.utils import direction_to, my_turn_to_place_factory
import numpy as np
import sys
class Agent():
    def __init__(self, player: str, env_cfg: EnvConfig) -> None: # env_cfg = Lux 게임의 환경 설정을 담은 객체
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0" # player 설정 단계
        np.random.seed(0) # 랜덤 시드 고정
        self.env_cfg: EnvConfig = env_cfg 
        
        # self.faction_names = 각 플레이어 이름에 해당하는 진영 이름을 나타냄    
        self.faction_names = {              
            'player_0': 'TheBuilders',
            'player_1': 'FirstMars' 
        }
        
        self.bots = {}
        self.botpos = []
        self.bot_factory = {}
        self.factory_bots = {}
        self.factory_queue = {}
        self.move_deltas = np.array([[0, 0], [0, -1], [1, 0], [0, 1], [-1, 0]]) # Lux 게임에서 유닛들이 이동할 때 사용되는 방향 벡터들을 담고 있는 배열
    
    # 게임의 초기 설정 단계 / 이 메소드는 'obs' = 현재 게임 상태 와 게임 플에이어가 할 수 있는 행동을 결정하는데 필요한 정보들을 사용하여 게임 플레이어의 행동을 반환 
    def early_setup(self, step: int, obs, remainingOverageTime: int = 60):
        '''
        Early Phase
        '''
        
        actions = dict() # 플레이어가 할 수 있는 행동을 담은 dic
        if step == 0:
            # Declare faction / 자신의 소속 진영과 입찰가를 설정
            actions['faction'] = self.faction_names[self.player]
            actions['bid'] = 0 # Learnable
        else: 
            # Factory placement period / 게임 시작 후 => 공장을 배치하는 단계
            # optionally convert observations to python objects with utility functions
            game_state = obs_to_game_state(step, self.env_cfg, obs) # obs를 게임 상태 객체인 'GameState'로 변환하여 저장
            opp_factories = [f.pos for _,f in game_state.factories[self.opp_player].items()] # 상대방의 공장 위치를 리스트에 저장
            my_factories = [f.pos for _,f in game_state.factories[self.player].items()] # 자신의 공장 위치를 리스트에 저장
            
            # how much water and metal you have in your starting pool to give to new factories / 남아있는 물과 금속의 양을 저장
            water_left = game_state.teams[self.player].water 
            metal_left = game_state.teams[self.player].metal
            
            # how many factories you have left to place
            factories_to_place = game_state.teams[self.player].factories_to_place # 배치할 수 있는 공장의 개수
            my_turn_to_place = my_turn_to_place_factory(game_state.teams[self.player].place_first, step) # 자신의 차례인지 여부를 저장
            if factories_to_place > 0 and my_turn_to_place:
                # we will spawn our factory in a random location with 100 metal n water (learnable)
                potential_spawns = np.array(list(zip(*np.where(obs["board"]["valid_spawns_mask"] == 1)))) # 랜덤한 위치에 공장을 배치하기 위해 사용할 수 있는 모든 위치를 리스트에 저장
                
                
                # 현재 얼음과 광물이 존재하는 타일의 위치를 각각 저장
                ice_map = game_state.board.ice
                ore_map = game_state.board.ore
                
                # 얼음과 광물이 존재하는 모든 타일의 위치를 각각 저장
                ice_tile_locations = np.argwhere(ice_map == 1) 
                ore_tile_locations = np.argwhere(ore_map == 1)
                
                # 현재까지 찾은 최소 거리 값을 저장할 변수                 
                min_dist = 10e6 # 일부러 지극히 큰 값을 줌으로써, 실제 거리값이 이보다 작으면 그것이 최소값이 될 수 있기 때문이다.
                
                # 현재까지 찾은 최소거리를 갖는 위치를 저장할 변수
                best_loc = potential_spawns[0]
                
                # 공장 배치 위치 주변의 rubble 밀도 계산에 사용할 변수
                d_rubble = 10
                
                for loc in potential_spawns:
                    
                    ice_tile_distances = np.mean((ice_tile_locations - loc) ** 2, 1) # 각 얼음 타일과 현재 위치 간의 거리를 계산
                    ore_tile_distances = np.mean((ore_tile_locations - loc) ** 2, 1) # 각 광물 타일과 현재 위치 간의 거리를 계산
                    
                    # obs["board"]["rubble"]는 48*48 배열로 나타남
                    density_rubble = np.mean(obs["board"]["rubble"][max(loc[0]-d_rubble,0):min(loc[0]+d_rubble,47), max(loc[1]-d_rubble,0):max(loc[1]+d_rubble,47)]) # 현재 위치 주변의 쓰레기 밀도를 계산
                    
                    
                    # 적군 공장과 현재 위치 간의 거리 중 최소 거리를 계산
                    closes_opp_factory_dist = 0
                    if len(opp_factories) >= 1: # 적군 공장이 1개 이상일 때
                        closes_opp_factory_dist = np.min(np.mean((np.array(opp_factories) - loc)**2, 1))
                        
                    # 아군 공장과 현재 위치 간의 거리 중 최소 거리를 계산
                    closes_my_factory_dist = 0
                    if len(my_factories) >= 1:
                        closes_my_factory_dist = np.min(np.mean((np.array(my_factories) - loc)**2, 1))
                        
                    # np.min(ice_tile_distances): 얼음 타일과 후보 위치 간 거리 중 가장 작은 값을 10배하여 반환합니다. 이 값을 작게 만들어야 더 가까운 위치를 선호
                    # 0.01*np.min(ore_tile_distances): 광물 타일과 후보 위치 간 거리 중 가장 작은 값을 0.01배하여 반환합니다. 이 값을 크게 만들어야 더 많은 광물을 채취할 수 있는 위치를 선호
                    # 10*density_rubble/(d_rubble): 후보 위치의 주변에 존재하는 미네랄 타일의 밀도를 반환합니다. 밀도는 타일의 개수 대비 미네랄이 차지하는 비율을 의미합니다. 이 값을 크게 만들어야 더 많은 미네랄을 채취할 수 있는 위치를 선호
                    # closes_opp_factory_dist*0.1 : 상대방 공장과 후보 위치 간 거리 중 가장 가까운 거리를 0.1배하여 반환, 이 값을 작게 만들어야 상대방의 공격을 받을 가능성이 적은 위치를 선호
                    # minimum_ice_dist: 가장 작은 값을 가지는 위치가 가장 적합한 위치로 간주되어, 해당 위치에 유닛을 생성
                    
                    # 10, 0.01, 0.1의 의미
                    # 중요도의 차이
                    # 얼음: 가까운 얼음 타일을 우선적으로 선택하도록 유도
                    # 광석: 게임에서 높은 가치를 지닌 리소스, But 많은 시간이 걸리고 리소스가 많이 들어가서 채취하기 어렵다. 얼음 타일에 우선적으로 집중하기 위해 가중치 0.01을 곱함
                    # 잔해: 주변 잔해 밀도가 낮은 지역이 선택되도록 유도
                    
                    
                    minimum_ice_dist = np.min(ice_tile_distances)*10 + 0.01*np.min(ore_tile_distances) + 10*density_rubble/(d_rubble) - closes_opp_factory_dist*0.1 + closes_opp_factory_dist*0.01
                    # 위에 작성된 식은 적군 공장과의 거리가 가까울 수록 더 위험하다고 가정하여, 적군 공장과의 거리를 더 크게 패널티로 부여하기 위해서 빼는 식이다.
                    if minimum_ice_dist < min_dist:
                        min_dist = minimum_ice_dist
                        best_loc = loc
                
#                 spawn_loc = potential_spawns[np.random.randint(0, len(potential_spawns))]
                spawn_loc = best_loc
                actions['spawn']=spawn_loc
#                 actions['metal']=metal_left
#                 actions['water']=water_left
                actions['metal']=min(300, metal_left) 
                actions['water']=min(300, water_left)
            
        return actions
    
    def check_collision(self, pos, direction, unitpos, unit_type = 'LIGHT'):
        move_deltas = np.array([[0, 0], [0, -1], [1, 0], [0, 1], [-1, 0]]) # 상, 하, 좌, 우, 제자리
#         move_deltas = np.array([[0, 0], [-1, 0], [0, 1], [1, 0], [0, -1]])
        
        new_pos = pos + move_deltas[direction]
        
         
        if unit_type == "LIGHT": # 경량 로봇일 경우
            return str(new_pos) in unitpos or str(new_pos) in self.botposheavy.values() # unitpos = 'Light', botposheavy = 'Heavy'
        else:
            return str(new_pos) in unitpos
    
    def get_direction(self,unit, closest_tile, sorted_tiles):
        
        closest_tile = np.array(closest_tile)
        direction = direction_to(np.array(unit.pos), closest_tile) # unit의 현재 위치와 가장 가까운 타일 사이의 방향을 계산
        k=0
        all_unit_positions = set(self.botpos.values())
        unit_type = unit.unit_type
        while self.check_collision(np.array(unit.pos), direction, all_unit_positions, unit_type) and k < min(len(sorted_tiles)-1, 500):
            k += 1
            closest_tile = sorted_tiles[k]
            closest_tile = np.array(closest_tile)
            direction = direction_to(np.array(unit.pos), closest_tile)
        
        if self.check_collision(unit.pos, direction, all_unit_positions, unit_type):
            for direction_x in np.arange(4,-1,-1):
                if not self.check_collision(np.array(unit.pos), direction_x, all_unit_positions, unit_type): # 충돌하지 않는 방향을 찾을 때까지 모든 방향을 시도
                    direction = direction_x
                    break

        if self.check_collision(np.array(unit.pos), direction, all_unit_positions, unit_type):
            direction = np.random.choice(np.arange(5))
            
        move_deltas = np.array([[0, 0], [0, -1], [1, 0], [0, 1], [-1, 0]]) # 이동 방향을 결정할 때 사용될 상대적인 방향들을 정의한 numpy 배열을 생성
        
        self.botpos[unit.unit_id] = str(np.array(unit.pos) + move_deltas[direction])
        # np.array(unit.pos) == numpy 배열로 표현된 유닛의 위치
        # 딕셔너리의 키 값은 문자열이어야 한다. 그래서 유닛의 위치를 문자열로 바꿔서 저장
        
        return direction

    def act(self, step: int, obs, remainingOverageTime: int = 60):
        '''
        1. Regular Phase
        2. Building Robots
        '''
        actions = dict()
        game_state = obs_to_game_state(step, self.env_cfg, obs) # env_cfg = 맵의 크기, 유닛 종류, 유닛의 최대 수등과 같은 환경의 정적인 설정 정보, 게임이 시작될 때 정해지는 환경 설정값
        # obs: 게임이 진행되는 동안 변경되는 게임 상황 정보
        state_obs = obs
        
        # Unit locations
        self.botpos = {}
        self.botposheavy = {}
        self.opp_botpos = []
        for player in [self.player, self.opp_player]:
            for unit_id, unit in game_state.units[player].items():
                
                if player == self.player:
                    self.botpos[unit_id] = str(unit.pos) # unit.pos: 현재 유닛의 위치를 나타내는 변수 / botpos: 현재 봇이 가지고 있는 모든 유닛들의 위치 정보를 담은 딕셔너리
                else:
                    self.opp_botpos.append(unit.pos) 
                    
                
                if unit.unit_type == "HEAVY": # heavy 로봇의 위치 정보만 따로 추려서 저장.
                    self.botposheavy[unit_id] = str(unit.pos)
        
        # Build Robots
        factories = game_state.factories[self.player]
        factory_tiles, factory_units, factory_ids = [], [], []
        bot_units = {}
        
        for unit_id, factory in factories.items(): # unit_id = 해당 공장을 나타냄, factory: 해당 공장의 상태(ex. 공장의 현재 수용 가능한 로봇 수, 현재 생산 가능한 로봇 종류, 생산에 필요한 자원)
            
            if unit_id not in self.factory_bots.keys(): # self.factory_bots.keys()를 호출 시, 해당 플레이어가 소유한 모든 공장들의 unit_id를 리스트 형태로 반환
                self.factory_bots[unit_id] = {
                    'ice':[],
                    'ore':[],
                    'rubble':[],
                    'kill':[],
                }
                
                self.factory_queue[unit_id] = []
            
            for task in ['ice', 'ore', 'rubble', 'kill']: # 다양한 로봇 타입
                for bot_unit_id in self.factory_bots[unit_id][task]:
                    if bot_unit_id not in self.botpos.keys(): # 로봇들의 전체 위치 중에서 ~~
                        self.factory_bots[unit_id][task].remove(bot_unit_id)
            
            minbot_task = None
            min_bots = {        # 각각의 로봇 타입과 필요한 최소한의 로봇 수를 지정
                'ice':1,
                'ore':5,
                'rubble':5,
                'kill':1
            }
            # NO. BOTS PER TASK
            for task in ['kill', 'ice', 'ore', 'rubble']:
                num_bots = len(self.factory_bots[unit_id][task]) + sum([task in self.factory_queue[unit_id]])
                if num_bots < min_bots[task]:
                    minbots = num_bots
                    minbot_task = task
                    break
            # 각 공장에 로봇을 추가로 생산하는 코드 / 전략을 새로 짜야할 듯      
            if minbot_task is not None:
                if minbot_task in ['kill', 'ice']:
                    if factory.power >= self.env_cfg.ROBOTS["HEAVY"].POWER_COST and \
                    factory.cargo.metal >= self.env_cfg.ROBOTS["HEAVY"].METAL_COST: # factory에 쌓아둔 메탈
                        actions[unit_id] = factory.build_heavy()
                    elif factory.power >= self.env_cfg.ROBOTS["LIGHT"].POWER_COST and \
                    factory.cargo.metal >= self.env_cfg.ROBOTS["LIGHT"].METAL_COST:
                        actions[unit_id] = factory.build_light()
                else: # ore, rubble
                    if factory.power >= self.env_cfg.ROBOTS["LIGHT"].POWER_COST and \
                    factory.cargo.metal >= self.env_cfg.ROBOTS["LIGHT"].METAL_COST:
                        actions[unit_id] = factory.build_light()
                    elif factory.power >= self.env_cfg.ROBOTS["HEAVY"].POWER_COST and \
                    factory.cargo.metal >= self.env_cfg.ROBOTS["HEAVY"].METAL_COST:
                        actions[unit_id] = factory.build_heavy()
                
                # 현재 생산 중인 로봇이 없을 때, 생산할 로봇의 종류를 추가         
                if unit_id not in self.factory_queue.keys():
                    self.factory_queue[unit_id] = [minbot_task]
                else:
                    self.factory_queue[unit_id].append(minbot_task)#  ....?

            factory_tiles += [factory.pos] # 공장의 위치를 저장하는 리스트
            factory_units += [factory] # 공장 객체를 저장하는 리스트 ex. factory_units[0] = 맵 상의 첫 번째 공장 객체를 가리킴
            factory_ids += [unit_id] # 공장의 ID를 저장하는 리스트
            
            if factory.can_water(game_state) and step > 900 and factory.cargo.water > (1000-step)+100:
                actions[unit_id] = factory.water()
        
        factory_tiles = np.array(factory_tiles) # Factory locations (to go back to)
            
        # Move Robots
        # iterate over our units and have them mine the closest ice tile # 가장 가까운 얼음 타일을 채굴
        units = game_state.units[self.player]
        
        # Resource map and locations
        ice_map = game_state.board.ice
        ore_map = game_state.board.ore
        rubble_map = game_state.board.rubble
        
        # 특정 데이터의 위치를 찾기
        ice_locations_all = np.argwhere(ice_map >= 1) # numpy position of every ice tile
        ore_locations_all = np.argwhere(ore_map >= 1) # numpy position of every ore tile
        rubble_locations_all = np.argwhere(rubble_map >= 1) # numpy position of every rubble tile
        
        ice_locations = ice_locations_all
        ore_locations = ore_locations_all
        rubble_locations = rubble_locations_all
            
        for unit_id, unit in iter(sorted(units.items())):
            
            if unit_id not in self.bots.keys(): # 공장이 없다면??
                self.bots[unit_id] = ''
                
            if len(factory_tiles) > 0:
                closest_factory_tile = factory_tiles[0]
            
            if unit_id not in self.bot_factory.keys(): # 해당 로봇이 현재 매핑된 공장이 없다면, 새로운 공장 중 가까운 공장을 찾아서 self.bot_factory 딕셔너리에 추가
                factory_distances = np.mean((factory_tiles - unit.pos) ** 2, 1)
                min_index = np.argmin(factory_distances)
                closest_factory_tile = factory_tiles[min_index] 
                self.bot_factory[unit_id] = factory_ids[min_index] # 가장 가까운 공장의 위치 정보와 해당 공장의 unit_id를 self.bot_factory 딕셔너리에 추가
            elif self.bot_factory[unit_id] not in factory_ids: # 해당 로봇이 매핑된 공장이 있지만, 더 이상 유효하지 않은 공장과 매핑된 경우, 다시 가까운 공장을 찾아서 매핑을 수정해줌
                factory_distances = np.mean((factory_tiles - unit.pos) ** 2, 1)
                min_index = np.argmin(factory_distances)
                closest_factory_tile = factory_tiles[min_index]
                self.bot_factory[unit_id] = factory_ids[min_index]
            else:
                closest_factory_tile = factories[self.bot_factory[unit_id]].pos
                
                
            distance_to_factory = np.mean((np.array(closest_factory_tile) - np.array(unit.pos))**2)
            adjacent_to_factory = False
            sorted_factory = [closest_factory_tile] # 초기값으로 해당 로봇이 가장 가까운 공장의 위치를 리스트에 담아둠
             
            if unit.power < unit.action_queue_cost(game_state): #해당 로봇의 현재 에너지가 행동을 하기에 충분한지 확인하고 충분하지 않다면 다음 유닛을 처리
                continue # 상대방이 나의 자원 생산을 막기 위해 많은 로봇들을 내보내고 있을 때, 내 로봇들도 많이 움직여야 하지만, 모든 로봇들이 움직일 때마다 에너지가 소모. 현재 에너지가 충분하지 않은 로봇은 행동 X
                
            if len(factory_tiles) > 0:
                
                move_cost = None
                try:
                    adjacent_to_factory = np.mean((np.array(closest_factory_tile) - np.array(unit.pos)) ** 2) <= 1 # 현재 로봇과 가장 가까운 공장의 위치가 유효한지 확인
                except:
                    print(closest_factory_tile, unit.pos) 
                    assert False 
                
                
                # 공장의 대기열 = 해당 공장에서 생성된 로봇이 수행할
                ## Assigning task for the bot
                if self.bots[unit_id] == '':
                    task = 'ice'
                    if len(self.factory_queue[self.bot_factory[unit_id]]) != 0: # 해당 로봇이 속한 공장의 대기열에 작업이 있다면
                        task = self.factory_queue[self.bot_factory[unit_id]].pop(0) # task 변수에 해당 공장의 대기열에서 첫 번째 작업을 가져온다.
                    self.bots[unit_id] = task
                    self.factory_bots[self.bot_factory[unit_id]][task].append(unit_id) # 해당 로봇이 속한 공장에서 수행할 작업에 대한 작업자 목록에 해당 로봇을 추가
                
                battery_capacity = 150 if unit.unit_type == "LIGHT" else 3000
                cargo_space = 100 if unit.unit_type == "LIGHT" else 1000
                def_move_cost = 1 if unit.unit_type == "LIGHT" else 20
                rubble_dig_cost = 5 if unit.unit_type == "LIGHT" else 100
                
                
                # cargo_space = 얼음을 저장할 수 있는 최대 공간 / 
                # unit.action_queue_cost(game_state): 현재 로봇의 행동 대기열에 있는 모든 동작의 비용 합계를 반환하는 함수 ex) 로봇이 move와 dig를 대기열에 추가한 경우 두 행동의 비용의 합
                if self.bots[unit_id] == "ice":
                    if unit.cargo.ice < cargo_space and unit.power > unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + def_move_cost*distance_to_factory:

                        # compute the distance to each ice tile from this unit and pick the closest
                        
                        ice_rubbles = np.array([rubble_map[pos[0]][pos[1]] for pos in ice_locations])
                        ice_distances = np.mean((ice_locations - unit.pos) ** 2, 1) #- (ice_rubbles)*10
                        sorted_ice = [ice_locations[k] for k in np.argsort(ice_distances)]
                        
                        closest_ice = sorted_ice[0]
                        # if we have reached the ice tile, start mining if possible
                        if np.all(closest_ice == unit.pos): # np.all(조건) - 배열의 모든 데이터가 조건과 맞으면 True 하나라도 다르면 False
                            if unit.power >= unit.dig_cost(game_state) +\
                            unit.action_queue_cost(game_state):
                                actions[unit_id] = [unit.dig(repeat=False)] # actions = dictionary 형태
                        else:
                            direction = self.get_direction(unit, closest_ice, sorted_ice)
                            move_cost = unit.move_cost(game_state, direction)

                    elif unit.cargo.ice >= cargo_space or unit.power <= unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + def_move_cost*distance_to_factory:

                        if adjacent_to_factory:
                            # 공장 타일이 (0,0)이라고 가정한 것은 이 코드의 작성자가 경기 맵의 특정 상황에서 공장 타일이 항상 맨 왼쪽 상단에 위치하고 있다고 가정한 결과. 따라서 이 코드에서는
                            # 모든 로봇이 항상 (0,0) 타일에서 가장 가까운 타일로 이동. 하지만 경기 맵이 바뀌거나, 게임 규칙이 변경될 경우 이 코드가 제대로 작동하지 않을 수 있다.
                            
                            if unit.cargo.ice > 0:
                                actions[unit_id] = [unit.transfer(0, 0, unit.cargo.ice, repeat=False)]
                            elif unit.cargo.ore > 0:
                                actions[unit_id] = [unit.transfer(0, 1, unit.cargo.ore, repeat=False)]
                            elif unit.power < battery_capacity*0.1:
                                actions[unit_id] = [unit.pickup(4, battery_capacity-unit.power)] # 4가 전력을 의미??
                        else: # 로봇이 공장 근처에 없는 경우, 다음을 실행
                            direction = self.get_direction(unit, closest_factory_tile, sorted_factory)
                            move_cost = unit.move_cost(game_state, direction)
                            
                elif self.bots[unit_id] == 'ore':
                    if unit.cargo.ore < cargo_space and unit.power > unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + def_move_cost*distance_to_factory:

                        # compute the distance to each ore tile from this unit and pick the closest
                        # rubble_map = 2D numpy array // pos[0]= x좌표, pos[1] = y좌표
                        ore_rubbles = np.array([rubble_map[pos[0]][pos[1]] for pos in ore_locations])
                        ore_distances = np.mean((ore_locations - unit.pos) ** 2, 1) #+ (ore_rubbles)*2
                        sorted_ore = [ore_locations[k] for k in np.argsort(ore_distances)]
                        
                        closest_ore = sorted_ore[0]
                        # if we have reached the ore tile, start mining if possible
                        if np.all(closest_ore == unit.pos):
                            if unit.power >= unit.dig_cost(game_state) +\
                            unit.action_queue_cost(game_state):
                                actions[unit_id] = [unit.dig(repeat=False)]
                        else:
                            direction = self.get_direction(unit, closest_ore, sorted_ore)
                            move_cost = unit.move_cost(game_state, direction)

                    elif unit.cargo.ore >= cargo_space or unit.power <= unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + def_move_cost*distance_to_factory:

                        if adjacent_to_factory:
                            if unit.cargo.ore > 0:
                                actions[unit_id] = [unit.transfer(0, 1, unit.cargo.ore, repeat=False)]
                            elif unit.cargo.ice > 0:
                                actions[unit_id] = [unit.transfer(0, 0, unit.cargo.ice, repeat=False)]
                            elif unit.power < battery_capacity*0.1:
                                actions[unit_id] = [unit.pickup(4, battery_capacity-unit.power)]
                        else:
                            direction = self.get_direction(unit, closest_factory_tile, sorted_factory)
                            move_cost = unit.move_cost(game_state, direction)
                elif self.bots[unit_id] == 'rubble':
                    if unit.power > unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + rubble_dig_cost:

                        # compute the distance to each rubble tile from this unit and pick the closest
                        rubble_distances = np.mean((rubble_locations - unit.pos) ** 2, 1)
                        sorted_rubble = [rubble_locations[k] for k in np.argsort(rubble_distances)]
                        closest_rubble = sorted_rubble[0]

                        # if we have reached the rubble tile, start mining if possible
                        if np.all(closest_rubble == unit.pos) or rubble_map[unit.pos[0], unit.pos[1]] != 0:
                            if unit.power >= unit.dig_cost(game_state) +\
                            unit.action_queue_cost(game_state):
                                actions[unit_id] = [unit.dig(repeat=False)]
                        else:
                            if len(rubble_locations) != 0:
                                direction = self.get_direction(unit, closest_rubble, sorted_rubble)
                                move_cost = unit.move_cost(game_state, direction)

                    elif unit.power <= unit.action_queue_cost(game_state) + unit.dig_cost(game_state) + rubble_dig_cost:

                        if adjacent_to_factory:
                            if unit.cargo.ore > 0:
                                actions[unit_id] = [unit.transfer(0, 1, unit.cargo.ore, repeat=False)]
                            elif unit.cargo.ice > 0:
                                actions[unit_id] = [unit.transfer(0, 0, unit.cargo.ice, repeat=False)]
                            elif unit.power < battery_capacity*0.1:
                                actions[unit_id] = [unit.pickup(4, battery_capacity-unit.power)]
                        else:
                            direction = self.get_direction(unit, closest_factory_tile, sorted_factory)
                            move_cost = unit.move_cost(game_state, direction)
                elif self.bots[unit_id] == 'kill':
                    
                    if len(self.opp_botpos) != 0:
                        opp_pos = np.array(self.opp_botpos).reshape(-1,2)
                        opponent_unit_distances = np.mean((opp_pos - unit.pos)**2,1)
                        min_distance = np.min(opponent_unit_distances)
                        pos_min_distance = opp_pos[np.argmin(min_distance)]

                        if min_distance == 1:
                            direction = self.get_direction(unit, np.array(pos_min_distance), [np.array(pos_min_distance)])
                            move_cost = unit.move_cost(game_state, direction)
                        else:
                            if unit.power > unit.action_queue_cost(game_state):
                                direction = self.get_direction(unit, np.array(pos_min_distance), [np.array(pos_min_distance)])
                                move_cost = unit.move_cost(game_state, direction)
                            else:
                                if adjacent_to_factory:
                                    if unit.cargo.ore > 0:
                                        actions[unit_id] = [unit.transfer(0, 1, unit.cargo.ore, repeat=False)]
                                    elif unit.cargo.ice > 0:
                                        actions[unit_id] = [unit.transfer(0, 0, unit.cargo.ice, repeat=False)]
                                    elif unit.power < battery_capacity*0.1:
                                        actions[unit_id] = [unit.pickup(4, battery_capacity-unit.power)]
                                else:
                                    direction = self.get_direction(unit, closest_factory_tile, sorted_factory)
                                    move_cost = unit.move_cost(game_state, direction)

                # check move_cost is not None, meaning that direction is not blocked
                # check if unit has enough power to move and update the action queue.
                if move_cost is not None and unit.power >= move_cost + unit.action_queue_cost(game_state):
                    actions[unit_id] = [unit.move(direction, repeat=False)]

         
        return actions

In [14]:
!luxai-s2 main.py main.py -v 2 -s 42 -o replay.html

790: player_1 lost all factories
25.75396990776062


In [15]:
import os
os.listdir()

['.gitignore',
 'agent.py',
 'lux',
 'lux-ai-challenge-season-2-tutorial-python.ipynb',
 'main.py',
 'README.md',
 'replay.html',
 '__pycache__']

In [ ]:
HTML(f"""<iframe src=replay.html width=1040 height=560 frameBorder="0" id="luxEye2022IFrame{get_ipython().execution_count}"></iframe>""")